Scripts

In [1]:
from newsapi import NewsApiClient
import datetime
import time
import pandas as pd
import os
from newspaper import Article
import sys

In [ ]:
# Initialize NewsAPI Client
api_key = ""  # Replace with API key
newsapi = NewsApiClient(api_key=api_key)

# Input and output file paths
input_file = "news_articles_election_candidates_expanded.csv"
full_content_file = "news_articles_election_candidates_full_content.csv"

# Function to fetch full article content using newspaper3k
def fetch_full_content(article_url):
    try:
        article = Article(article_url)
        article.download()
        article.parse()
        return article.text  # Return the full article text
    except Exception as e:
        return None  # Return None if there is an error

# Set new date range
start_date = datetime.date(2024, 10, 12)  # Continue from where the previous script left off
end_date = datetime.date(2024, 8, 1)  # Adjust end date as needed for backward collection

# Prepare to store results
articles_data = []

# Expanded search queries
queries = [
    "2024 Presidential election",
    "US election AND (Donald Trump OR Kamala Harris)",
    "Biden administration AND 2024 election",
    "(Donald Trump OR Trump) AND 2024 election",
    "(Kamala Harris OR Harris) AND 2024 election",
    "Campaign financing AND 2024 election",
    "Voter turnout AND 2024 election",
    "Presidential debate AND 2024 election",
    "(Donald Trump OR Trump) AND rally AND 2024",
    "(Kamala Harris OR Harris) AND speech AND 2024",
    "(Donald Trump OR Kamala Harris) AND 2024 election",
    "(Trump OR Harris) AND campaign AND 2024 election"
]

# Track API request count to avoid exceeding limits
request_count = 0
max_requests = 100  # Free-tier daily API limit

# Load existing expanded and full content CSV files
existing_expanded_data = pd.read_csv(input_file) if os.path.exists(input_file) else pd.DataFrame()
existing_full_content_data = pd.read_csv(full_content_file) if os.path.exists(full_content_file) else pd.DataFrame()

# Track URLs to avoid duplicates
existing_urls = set(existing_expanded_data["url"]) if not existing_expanded_data.empty else set()
processed_urls = set(existing_full_content_data["url"]) if not existing_full_content_data.empty else set()

# Fetch new articles from NewsAPI
current_date = start_date
while current_date >= end_date:
    # Convert date to string for API
    date_str = current_date.strftime('%Y-%m-%d')
    print(f"Fetching articles for {date_str}...")

    for query in queries:
        try:
            # Check if API limit is reached
            if request_count >= max_requests:
                print("Reached API limit for the day. Exiting script.")
                sys.exit()

            # Fetch articles for the current query and date
            response = newsapi.get_everything(
                q=query,
                from_param=date_str,
                to=date_str,
                language="en",
                sort_by="relevancy",  # Fetch relevant articles
                page_size=100  # Max articles per API call
            )

            # Increment request count
            request_count += 1

            if response.get('status') != 'ok':
                print(f"API error: {response.get('message')}")
                sys.exit()

            # Process the articles
            if response.get('articles'):
                for article in response['articles']:
                    # Only add new articles that are not already saved
                    if article['url'] not in existing_urls:
                        articles_data.append({
                            "query": query,  # Include the query used for tracking
                            "source": article['source']['name'],
                            "author": article['author'],
                            "title": article['title'],
                            "description": article['description'],
                            "url": article['url'],
                            "published_at": article['publishedAt'],
                            "content": article['content']
                        })
                        # Add the URL to the set of existing URLs
                        existing_urls.add(article['url'])
        except Exception as e:
            # Log the error to a file
            with open("error_log.txt", "a") as log_file:
                log_file.write(f"Error fetching articles for {query} on {date_str}: {e}\n")
            print(f"Error fetching articles for {query} on {date_str}: {e}")
    
    # Move to the previous day
    current_date -= datetime.timedelta(days=1)

    # Avoid hitting API limits by adding a small delay between requests
    time.sleep(1)

# Save new articles to the expanded CSV file
if articles_data:
    new_data_df = pd.DataFrame(articles_data)
    new_data_df.to_csv(input_file, mode='a', header=not os.path.exists(input_file), index=False)
    print(f"Appended {len(new_data_df)} new articles to '{input_file}'.")

# Extract full content for new articles only
new_urls = {article["url"] for article in articles_data}  # URLs of newly fetched articles
urls_to_process = new_urls - processed_urls  # Exclude already processed URLs

if urls_to_process:
    full_content_data = []
    for url in urls_to_process:
        print(f"Fetching full content for {url}...")
        full_content = fetch_full_content(url)
        if full_content:
            full_content_data.append({
                "url": url,
                "full_content": full_content
            })
        else:
            print(f"Failed to fetch full content for {url}")

    # Append full content to the full content CSV file
    if full_content_data:
        full_content_df = pd.DataFrame(full_content_data)
        full_content_df.to_csv(full_content_file, mode='a', header=not os.path.exists(full_content_file), index=False)
        print(f"Appended {len(full_content_df)} new full content articles to '{full_content_file}'.")
else:
    print("No new articles to process for full content.")



In [ ]:
# Load the CSV file
file_path = 'news_articles_election_candidates_expanded.csv'
data = pd.read_csv(file_path)

data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9610 entries, 0 to 9609
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   query         9610 non-null   object
 1   source        9610 non-null   object
 2   author        9093 non-null   object
 3   title         9607 non-null   object
 4   description   9598 non-null   object
 5   url           9610 non-null   object
 6   published_at  9610 non-null   object
 7   content       9610 non-null   object
dtypes: object(8)
memory usage: 600.8+ KB


In [ ]:
file_path = 'news_articles_election_candidates_full_content.csv'
data_full_content = pd.read_csv(file_path)

data_full_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9610 entries, 0 to 9609
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   query         9610 non-null   object
 1   source        9610 non-null   object
 2   author        9093 non-null   object
 3   title         9607 non-null   object
 4   description   9598 non-null   object
 5   url           9610 non-null   object
 6   published_at  9610 non-null   object
 7   content       9610 non-null   object
 8   full_content  7556 non-null   object
dtypes: object(9)
memory usage: 675.8+ KB


Remove entries with missing full content

START

In [2]:
data_full_content = pd.read_csv('news_articles_election_candidates_full_content_cleaned.csv')

In [8]:
data_full_content.head()

,query,source,author,title,description,url,published_at,content,full_content
0,2024 Presidential election,Wired,"Lily Hay Newman, Tess Owen",Russia Is Going All Out on Election Day Interf...,Along with other foreign influence operations—...,https://www.wired.com/story/russia-election-di...,2024-11-05T21:04:35Z,As the 2024 US presidential election comes to ...,As the 2024 US presidential election comes to ...
1,2024 Presidential election,The Verge,Justine Calma,Apple News will let you watch election results...,"On Election Day in the US, Apple News is rolli...",https://www.theverge.com/2024/11/5/24288777/el...,2024-11-05T16:34:12Z,Image: Cath Virginia / The Verge\r\n\n \n\n Fo...,For anyone obsessively watching election resul...
2,2024 Presidential election,NPR,Megan Pratz,Here's how NPR will report the 2024 election r...,NPR relies on results and race calls from The ...,https://www.npr.org/2024/11/04/g-s1-31268/2024...,2024-11-05T10:00:00Z,"Voters have been voting, ballots will be count...",Here's how NPR will report the 2024 election r...
3,2024 Presidential election,Business Insider,"insider@insider.com (John L. Dorman,Kelsey Vla...",The 2024 presidential election may come down t...,Election results in the swing states of Arizon...,https://www.businessinsider.com/what-are-2024-...,2024-11-05T22:13:18Z,"Over the past two decades, the road to the Whi...","On Election Day 2024, both parties are eyeing ..."
4,2024 Presidential election,CNET,Thomas Kika,How Do I Keep Track of Official Election Resul...,An avalanche of Election Day 2024 coverage is ...,https://www.cnet.com/tech/services-and-softwar...,2024-11-05T17:45:00Z,As if the 2024 election reporting could be any...,All anyone is talking about today is the presi...


In [6]:
print("Number of rows (articles):", data_full_content.shape[0])
print("Number of variables (columns):", data_full_content.shape[1])


Number of rows (articles): 7556
Number of variables (columns): 9


In [7]:
# Calculate the number of unique media outlets
unique_media_outlets = data_full_content['source'].nunique()

# Display the number of unique media outlets
print(f"Number of unique media outlets: {unique_media_outlets}")


Number of unique media outlets: 454


In [3]:
import networkx as nx
import spacy
from collections import defaultdict

# Process only the first 100 articles
data_sample = data_full_content.head(100)

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Initialize mappings for mentions
article_mentions = defaultdict(set)  # Maps article index to mentioned politicians
politician_mentions = defaultdict(set)  # Maps politician to articles they're mentioned in

# Perform NER to extract names of politicians dynamically
for idx, row in data_sample.iterrows():
    content = row['full_content']
    doc = nlp(content)
    for ent in doc.ents:
        if ent.label_ == "PERSON":  # Identify named entities classified as "PERSON"
            politician_name = ent.text
            article_mentions[idx].add(politician_name)
            politician_mentions[politician_name].add(idx)

# Create the network graph
G = nx.Graph()

# Add nodes with attributes (politicians and articles)
for politician, articles in politician_mentions.items():
    G.add_node(politician, articles=list(articles))

# Add edges based on co-mentions in articles
for article_idx, mentioned_politicians in article_mentions.items():
    mentioned_politicians = list(mentioned_politicians)
    for i in range(len(mentioned_politicians)):
        for j in range(i + 1, len(mentioned_politicians)):
            p1, p2 = mentioned_politicians[i], mentioned_politicians[j]
            if G.has_edge(p1, p2):
                G[p1][p2]['weight'] += 1
            else:
                G.add_edge(p1, p2, weight=1)

# Output graph information
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Sample nodes: {list(G.nodes(data=True))[:5]}")
print(f"Sample edges: {list(G.edges(data=True))[:5]}")

Number of nodes: 842
Number of edges: 34386
Sample nodes: [('Donald Trump', {'articles': [0, 1, 3, 4, 5, 6, 7, 9, 10, 12, 13, 15, 17, 18, 19, 20, 21, 24, 25, 27, 29, 30, 33, 34, 38, 40, 42, 43, 45, 49, 50, 51, 52, 53, 55, 57, 58, 64, 65, 67, 70, 71, 74, 75, 76, 77, 78, 82, 83, 84, 86, 91, 92, 93, 94, 95, 96, 97, 98, 99]}), ('Hillary Clinton', {'articles': [0, 33, 34, 3, 7, 12, 17, 49, 19, 82, 21, 92, 95]}), ('Clinton', {'articles': [0, 58, 3]}), ('Brad Raffensperger', {'articles': [0]}), ('Cait Conley', {'articles': [0]})]
Sample edges: [('Donald Trump', 'Brad Raffensperger', {'weight': 1}), ('Donald Trump', 'Tim Walz', {'weight': 7}), ('Donald Trump', 'Hillary Clinton', {'weight': 13}), ('Donald Trump', 'Adrian Fontes', {'weight': 2}), ('Donald Trump', 'Cait Conley', {'weight': 1})]


Combine e.g. hilary and hilary clinton
Make sure author is not a node e.g.